In [12]:
from pyspark import SparkContext
class FileReader:
    def __init__(self,dfile,rfile,cfile,xfile):
        self.data = self.getline(dfile)
        self.ridx = self.getline(rfile)
        self.cidx = self.getline(cfile)
        self.xvector = self.getline(xfile)
        self.ridx = [(i,self.ridx[i],self.ridx[i+1]) for i in range(len(self.ridx) -1)]

    def getline(self,fname):
        ifile = open(fname)
        values = ifile.readline().strip().split(' ')
        ifile.close()
        result = [int(v.strip()) for v in values]
        return result
    
    
class Mapper:
    def __init__(self):
        pass
    
    def map_func(self,a,b,fr):
        return [fr.data[j]*fr.xvector[fr.cidx[j]] for j in range(a,b,1)]
    
    def map(self,fr):
        sc = SparkContext.getOrCreate()
        ridx = sc.parallelize(fr.ridx)
        return ridx.map(lambda x:(x[0],self.map_func(x[1],x[2],fr)))
    
class Reducer:
    def __init__(self):
        pass
    
    def reduce(self,m):
        return m.map(lambda x: (x[0],sum(x[1]))).collect()
        
    
class Multiplicator:
    def __init__(self,dfile,rfile,cfile,xfile):
        fr = FileReader(dfile,rfile,cfile,xfile)
        mapper = Mapper()
        m = mapper.map(fr)
        reducer = Reducer()
        r = reducer.reduce(m)
        r = sorted(r,key=lambda x:x[0])
        result = ""
        for x in r:
            result += str(x[1]) + " "
        ofile = open("yvector",'w')
        ofile.write(result)
        ofile.close()
        
        
    
    
m = Multiplicator("data","rindx","cindx","xvector")
print("Done")

Done
